# Darts

In [ ]:
!pip install darts
!pip install dask[dataframe]

In [ ]:
!pip uninstall numpy
!pip install numpy
!pip install --force-reinstall darts

In [ ]:
import pandas as pd
import numpy as np
import darts
from darts import TimeSeries
import os
from scipy.special import comb
import matplotlib.pyplot as plt
import darts.models.forecasting
import time

In [ ]:
# os.chdir('/content/drive/MyDrive/Research/Bear_Island/Darts_one_step')

In [ ]:
import pandas as pd
df = pd.read_csv('/content/weekly_summary_1991-2021.csv')
print(df.head())

In [ ]:
# Creating train-test split
lags = 3
#colnames_dat = df.columns
dat = df.drop('Date', axis=1)
hor = 53
split_point = len(dat) - hor
# Split into train, validation, and test sets
tr_dat, ts_dat = dat[:split_point], dat[split_point:]
fore_hor = len(ts_dat)

In [ ]:
# Convert to Darts TimeSeries format
train_series = TimeSeries.from_dataframe(tr_dat)
test_series = TimeSeries.from_dataframe(ts_dat)

In [ ]:
# Create a folder by Equation name
folder_name = 'One_Step_Ahead_' + str(lags)
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    print(f"Folder '{folder_name}' created successfully.")
else:
    print(f"Folder '{folder_name}' already exists.")

Folder 'One_Step_Ahead_3' created successfully.


In [ ]:
from darts.metrics import metrics
smape_metric = {}
mase_metric = {}
mae_metric = {}
rmse_metric = {}
marre_metric = {}
model_train_pred_time = {}

## DLinearX

In [ ]:
# Define the DLinearModel Model
model_name = 'DLinearX'
model = darts.models.forecasting.dlinear.DLinearModel(input_chunk_length = lags, output_chunk_length = 1, random_state=42)
# Fit the model
start_time = time.time()
model.fit(train_series)
# Perform recursive forecasting for fore_hor steps
forecast = []
forecast_df = pd.DataFrame()
current_input = train_series[-lags:]  # Start with the last lags points
for i in range(fore_hor):
  pred = model.predict(n=1, series=current_input)
  forecast.append(pred)
  forecast_df = pd.concat([forecast_df, pred.to_dataframe()], ignore_index=True)
  current_input = current_input[1:]
  current_input = TimeSeries.from_dataframe(pd.concat([TimeSeries.to_dataframe(current_input), ts_dat.iloc[[i]]], ignore_index=True))
end_time = time.time()
smape_metric[model_name] = metrics.smape(TimeSeries.from_values(test_series.values()[:, 0]), TimeSeries.from_values(forecast_df['Precipitation'].values))
#mase_metric[model_name] = metrics.mase(TimeSeries.from_values(test_series.values()[:, 0]), TimeSeries.from_values(forecast_df['Precipitation'].values))
mae_metric[model_name] = metrics.mae(TimeSeries.from_values(test_series.values()[:, 0]), TimeSeries.from_values(forecast_df['Precipitation'].values))
rmse_metric[model_name] = metrics.rmse(TimeSeries.from_values(test_series.values()[:, 0]), TimeSeries.from_values(forecast_df['Precipitation'].values))
marre_metric[model_name] = metrics.marre(TimeSeries.from_values(test_series.values()[:, 0]), TimeSeries.from_values(forecast_df['Precipitation'].values))
file_name = model_name + '.xlsx'
forecast_file_path = os.path.join(folder_name, file_name)
forecast_df.to_excel(forecast_file_path, index=False)
print(f"Forecast saved to {forecast_file_path}")

## NLinearX

In [ ]:
# Define the NLinearModel Model
model_name = 'NLinearX'
model = darts.models.forecasting.nlinear.NLinearModel(input_chunk_length = lags, output_chunk_length = 1, random_state=42)
# Fit the model
start_time = time.time()
model.fit(train_series)
# Perform recursive forecasting for fore_hor steps
forecast = []
forecast_df = pd.DataFrame()
current_input = train_series[-lags:]  # Start with the last lags points
for i in range(fore_hor):
  pred = model.predict(n=1, series=current_input)
  forecast.append(pred)
  forecast_df = pd.concat([forecast_df, pred.to_dataframe()], ignore_index=True)
  current_input = current_input[1:]
  current_input = TimeSeries.from_dataframe(pd.concat([TimeSeries.to_dataframe(current_input), ts_dat.iloc[[i]]], ignore_index=True))
end_time = time.time()
smape_metric[model_name] = metrics.smape(TimeSeries.from_values(test_series.values()[:, 0]), TimeSeries.from_values(forecast_df['Precipitation'].values))
#mase_metric[model_name] = metrics.mase(TimeSeries.from_values(test_series.values()[:, 0]), TimeSeries.from_values(forecast_df['Precipitation'].values))
mae_metric[model_name] = metrics.mae(TimeSeries.from_values(test_series.values()[:, 0]), TimeSeries.from_values(forecast_df['Precipitation'].values))
rmse_metric[model_name] = metrics.rmse(TimeSeries.from_values(test_series.values()[:, 0]), TimeSeries.from_values(forecast_df['Precipitation'].values))
marre_metric[model_name] = metrics.marre(TimeSeries.from_values(test_series.values()[:, 0]), TimeSeries.from_values(forecast_df['Precipitation'].values))
file_name = model_name + '.xlsx'
forecast_file_path = os.path.join(folder_name, file_name)
forecast_df.to_excel(forecast_file_path, index=False)
print(f"Forecast saved to {forecast_file_path}")

## RandomForestX

In [ ]:
# Define the RandomForest Model
model_name = 'RandomForestX'
model = darts.models.forecasting.random_forest.RandomForest(lags = lags, output_chunk_length = 1, random_state=42)
# Fit the model
start_time = time.time()
model.fit(train_series)
# Perform recursive forecasting for fore_hor steps
forecast = []
forecast_df = pd.DataFrame()
current_input = train_series[-lags:]  # Start with the last lags points
for i in range(fore_hor):
  pred = model.predict(n=1, series=current_input)
  forecast.append(pred)
  forecast_df = pd.concat([forecast_df, pred.to_dataframe()], ignore_index=True)
  current_input = current_input[1:]
  current_input = TimeSeries.from_dataframe(pd.concat([TimeSeries.to_dataframe(current_input), ts_dat.iloc[[i]]], ignore_index=True))
end_time = time.time()
smape_metric[model_name] = metrics.smape(TimeSeries.from_values(test_series.values()[:, 0]), TimeSeries.from_values(forecast_df['Precipitation'].values))
#mase_metric[model_name] = metrics.mase(TimeSeries.from_values(test_series.values()[:, 0]), TimeSeries.from_values(forecast_df['Precipitation'].values))
mae_metric[model_name] = metrics.mae(TimeSeries.from_values(test_series.values()[:, 0]), TimeSeries.from_values(forecast_df['Precipitation'].values))
rmse_metric[model_name] = metrics.rmse(TimeSeries.from_values(test_series.values()[:, 0]), TimeSeries.from_values(forecast_df['Precipitation'].values))
marre_metric[model_name] = metrics.marre(TimeSeries.from_values(test_series.values()[:, 0]), TimeSeries.from_values(forecast_df['Precipitation'].values))
file_name = model_name + '.xlsx'
forecast_file_path = os.path.join(folder_name, file_name)
forecast_df.to_excel(forecast_file_path, index=False)
print(f"Forecast saved to {forecast_file_path}")

## LSTMX

In [ ]:
# Define the LSTM Model
model_name = 'LSTMX'
model = darts.models.forecasting.rnn_model.RNNModel(input_chunk_length = lags,
                                                    model='LSTM', random_state=42, training_length= lags+1)
# Fit the model
start_time = time.time()
model.fit(train_series)
# Perform recursive forecasting for fore_hor steps
forecast = []
forecast_df = pd.DataFrame()
current_input = train_series[-lags:]  # Start with the last lags points
for i in range(fore_hor):
  pred = model.predict(n=1, series=current_input)
  forecast.append(pred)
  forecast_df = pd.concat([forecast_df, pred.to_dataframe()], ignore_index=True)
  current_input = current_input[1:]
  current_input = TimeSeries.from_dataframe(pd.concat([TimeSeries.to_dataframe(current_input), ts_dat.iloc[[i]]], ignore_index=True))
end_time = time.time()
smape_metric[model_name] = metrics.smape(TimeSeries.from_values(test_series.values()[:, 0]), TimeSeries.from_values(forecast_df['Precipitation'].values))
#mase_metric[model_name] = metrics.mase(TimeSeries.from_values(test_series.values()[:, 0]), TimeSeries.from_values(forecast_df['Precipitation'].values))
mae_metric[model_name] = metrics.mae(TimeSeries.from_values(test_series.values()[:, 0]), TimeSeries.from_values(forecast_df['Precipitation'].values))
rmse_metric[model_name] = metrics.rmse(TimeSeries.from_values(test_series.values()[:, 0]), TimeSeries.from_values(forecast_df['Precipitation'].values))
marre_metric[model_name] = metrics.marre(TimeSeries.from_values(test_series.values()[:, 0]), TimeSeries.from_values(forecast_df['Precipitation'].values))
file_name = model_name + '.xlsx'
forecast_file_path = os.path.join(folder_name, file_name)
forecast_df.to_excel(forecast_file_path, index=False)
print(f"Forecast saved to {forecast_file_path}")

## NBeatsX

In [ ]:
# Final
model_name = 'NBEATSX'
model = darts.models.forecasting.nbeats.NBEATSModel(input_chunk_length=lags, output_chunk_length=1, random_state=42)
start_time = time.time()
# Fit the model
model.fit(train_series)
# Perform recursive forecasting for fore_hor steps
forecast = []
forecast_df = pd.DataFrame()
current_input = train_series[-lags:]  # Start with the last lags points
for i in range(fore_hor):
  pred = model.predict(n=1, series=current_input)
  forecast.append(pred)
  forecast_df = pd.concat([forecast_df, pred.to_dataframe()], ignore_index=True)
  current_input = current_input[1:]
  current_input = TimeSeries.from_dataframe(pd.concat([TimeSeries.to_dataframe(current_input), ts_dat.iloc[[i]]], ignore_index=True))
end_time = time.time()
smape_metric[model_name] = metrics.smape(TimeSeries.from_values(test_series.values()[:, 0]), TimeSeries.from_values(forecast_df['Precipitation'].values))
#mase_metric[model_name] = metrics.mase(TimeSeries.from_values(test_series.values()[:, 0]), TimeSeries.from_values(forecast_df['Precipitation'].values))
mae_metric[model_name] = metrics.mae(TimeSeries.from_values(test_series.values()[:, 0]), TimeSeries.from_values(forecast_df['Precipitation'].values))
rmse_metric[model_name] = metrics.rmse(TimeSeries.from_values(test_series.values()[:, 0]), TimeSeries.from_values(forecast_df['Precipitation'].values))
marre_metric[model_name] = metrics.marre(TimeSeries.from_values(test_series.values()[:, 0]), TimeSeries.from_values(forecast_df['Precipitation'].values))
file_name = model_name + '.xlsx'
forecast_file_path = os.path.join(folder_name, file_name)
forecast_df.to_excel(forecast_file_path, index=False)
print(f"Forecast saved to {forecast_file_path}")

## NHitsX

In [ ]:
# Define the NHiTSModel Model
model_name = 'NHiTSX'
model = darts.models.forecasting.nhits.NHiTSModel(input_chunk_length = lags, output_chunk_length = 1, random_state=42)
# Fit the model
start_time = time.time()
model.fit(train_series)
# Perform recursive forecasting for fore_hor steps
forecast = []
forecast_df = pd.DataFrame()
current_input = train_series[-lags:]  # Start with the last lags points
for i in range(fore_hor):
  pred = model.predict(n=1, series=current_input)
  forecast.append(pred)
  forecast_df = pd.concat([forecast_df, pred.to_dataframe()], ignore_index=True)
  current_input = current_input[1:]
  current_input = TimeSeries.from_dataframe(pd.concat([TimeSeries.to_dataframe(current_input), ts_dat.iloc[[i]]], ignore_index=True))
end_time = time.time()
smape_metric[model_name] = metrics.smape(TimeSeries.from_values(test_series.values()[:, 0]), TimeSeries.from_values(forecast_df['Precipitation'].values))
#mase_metric[model_name] = metrics.mase(TimeSeries.from_values(test_series.values()[:, 0]), TimeSeries.from_values(forecast_df['Precipitation'].values))
mae_metric[model_name] = metrics.mae(TimeSeries.from_values(test_series.values()[:, 0]), TimeSeries.from_values(forecast_df['Precipitation'].values))
rmse_metric[model_name] = metrics.rmse(TimeSeries.from_values(test_series.values()[:, 0]), TimeSeries.from_values(forecast_df['Precipitation'].values))
marre_metric[model_name] = metrics.marre(TimeSeries.from_values(test_series.values()[:, 0]), TimeSeries.from_values(forecast_df['Precipitation'].values))
file_name = model_name + '.xlsx'
forecast_file_path = os.path.join(folder_name, file_name)
forecast_df.to_excel(forecast_file_path, index=False)
print(f"Forecast saved to {forecast_file_path}")

## TransformerX

In [ ]:
# Define the TransformerModel Model
model_name = 'TransformerX'
model = darts.models.forecasting.transformer_model.TransformerModel(input_chunk_length = lags, output_chunk_length = 1, random_state=42)
# Fit the model
start_time = time.time()
model.fit(train_series)
# Perform recursive forecasting for fore_hor steps
forecast = []
forecast_df = pd.DataFrame()
current_input = train_series[-lags:]  # Start with the last lags points
for i in range(fore_hor):
  pred = model.predict(n=1, series=current_input)
  forecast.append(pred)
  forecast_df = pd.concat([forecast_df, pred.to_dataframe()], ignore_index=True)
  current_input = current_input[1:]
  current_input = TimeSeries.from_dataframe(pd.concat([TimeSeries.to_dataframe(current_input), ts_dat.iloc[[i]]], ignore_index=True))
end_time = time.time()
smape_metric[model_name] = metrics.smape(TimeSeries.from_values(test_series.values()[:, 0]), TimeSeries.from_values(forecast_df['Precipitation'].values))
#mase_metric[model_name] = metrics.mase(TimeSeries.from_values(test_series.values()[:, 0]), TimeSeries.from_values(forecast_df['Precipitation'].values))
mae_metric[model_name] = metrics.mae(TimeSeries.from_values(test_series.values()[:, 0]), TimeSeries.from_values(forecast_df['Precipitation'].values))
rmse_metric[model_name] = metrics.rmse(TimeSeries.from_values(test_series.values()[:, 0]), TimeSeries.from_values(forecast_df['Precipitation'].values))
marre_metric[model_name] = metrics.marre(TimeSeries.from_values(test_series.values()[:, 0]), TimeSeries.from_values(forecast_df['Precipitation'].values))
file_name = model_name + '.xlsx'
forecast_file_path = os.path.join(folder_name, file_name)
forecast_df.to_excel(forecast_file_path, index=False)
print(f"Forecast saved to {forecast_file_path}")

## TiDEX

In [ ]:
# Define the TiDEModel Model
model_name = 'TiDEX'
model = darts.models.forecasting.tide_model.TiDEModel(input_chunk_length = lags, output_chunk_length = 1, random_state=42)
# Fit the model
start_time = time.time()
model.fit(train_series)
# Perform recursive forecasting for fore_hor steps
forecast = []
forecast_df = pd.DataFrame()
current_input = train_series[-lags:]  # Start with the last lags points
for i in range(fore_hor):
  pred = model.predict(n=1, series=current_input)
  forecast.append(pred)
  forecast_df = pd.concat([forecast_df, pred.to_dataframe()], ignore_index=True)
  current_input = current_input[1:]
  current_input = TimeSeries.from_dataframe(pd.concat([TimeSeries.to_dataframe(current_input), ts_dat.iloc[[i]]], ignore_index=True))
end_time = time.time()
smape_metric[model_name] = metrics.smape(TimeSeries.from_values(test_series.values()[:, 0]), TimeSeries.from_values(forecast_df['Precipitation'].values))
#mase_metric[model_name] = metrics.mase(TimeSeries.from_values(test_series.values()[:, 0]), TimeSeries.from_values(forecast_df['Precipitation'].values))
mae_metric[model_name] = metrics.mae(TimeSeries.from_values(test_series.values()[:, 0]), TimeSeries.from_values(forecast_df['Precipitation'].values))
rmse_metric[model_name] = metrics.rmse(TimeSeries.from_values(test_series.values()[:, 0]), TimeSeries.from_values(forecast_df['Precipitation'].values))
marre_metric[model_name] = metrics.marre(TimeSeries.from_values(test_series.values()[:, 0]), TimeSeries.from_values(forecast_df['Precipitation'].values))
file_name = model_name + '.xlsx'
forecast_file_path = os.path.join(folder_name, file_name)
forecast_df.to_excel(forecast_file_path, index=False)
print(f"Forecast saved to {forecast_file_path}")

## XGBoostX

In [ ]:
# Define the XGBModel Model
model_name = 'XGBoostX'
model = darts.models.forecasting.xgboost.XGBModel(lags = lags, output_chunk_length = 1, random_state=42)
# Fit the model
start_time = time.time()
model.fit(train_series)
# Perform recursive forecasting for fore_hor steps
forecast = []
forecast_df = pd.DataFrame()
current_input = train_series[-lags:]  # Start with the last lags points
for i in range(fore_hor):
  pred = model.predict(n=1, series=current_input)
  forecast.append(pred)
  forecast_df = pd.concat([forecast_df, pred.to_dataframe()], ignore_index=True)
  current_input = current_input[1:]
  current_input = TimeSeries.from_dataframe(pd.concat([TimeSeries.to_dataframe(current_input), ts_dat.iloc[[i]]], ignore_index=True))
end_time = time.time()
smape_metric[model_name] = metrics.smape(TimeSeries.from_values(test_series.values()[:, 0]), TimeSeries.from_values(forecast_df['Precipitation'].values))
#mase_metric[model_name] = metrics.mase(TimeSeries.from_values(test_series.values()[:, 0]), TimeSeries.from_values(forecast_df['Precipitation'].values))
mae_metric[model_name] = metrics.mae(TimeSeries.from_values(test_series.values()[:, 0]), TimeSeries.from_values(forecast_df['Precipitation'].values))
rmse_metric[model_name] = metrics.rmse(TimeSeries.from_values(test_series.values()[:, 0]), TimeSeries.from_values(forecast_df['Precipitation'].values))
marre_metric[model_name] = metrics.marre(TimeSeries.from_values(test_series.values()[:, 0]), TimeSeries.from_values(forecast_df['Precipitation'].values))
file_name = model_name + '.xlsx'
forecast_file_path = os.path.join(folder_name, file_name)
forecast_df.to_excel(forecast_file_path, index=False)
print(f"Forecast saved to {forecast_file_path}")

## Metric Compile

In [ ]:
smape_metrics_df = pd.DataFrame.from_dict(smape_metric, orient='index', columns=['SMAPE'])\
rmse_metrics_df = pd.DataFrame.from_dict(rmse_metric, orient='index', columns=['RMSE'])
mae_metrics_df = pd.DataFrame.from_dict(mae_metric, orient='index', columns=['MAE'])
marre_metrics_df = pd.DataFrame.from_dict(marre_metric, orient='index', columns=['MARRE'])
execution_time_df = pd.DataFrame.from_dict(model_train_pred_time, orient='index', columns=['Time'])

In [ ]:
merged_df = pd.concat([smape_metrics_df, rmse_metrics_df, mae_metrics_df, marre_metrics_df, execution_time_df], axis=1)
merged_df